In [9]:
# %% [markdown]
# Wind Turbine Fault Detection — Full Notebook
#
# This Python script is formatted as notebook cells ("# %%" separators). Save as
# `wind_fault_full_notebook.py` and open in VSCode/Jupyter (Jupytext will convert
# to an .ipynb). It implements everything we ran: load data, binarize target,
# handle missing values, balance training set (SMOTE if available, else simple
# oversampling), feature selection (Mutual Information + XGBoost if available),
# train RandomForest, evaluate, visualize, save artifacts, and (optional) SHAP
# explanations.

# %%
# Imports
import os
import sys
import warnings
# import imblearn
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, auc
import matplotlib.pyplot as plt
# import joblib

# %% [markdown]
# ### Configuration — paths and options

# %%
# Paths (put Train.csv and Test.csv in the same folder as this script/notebook)
DATA_DIR = '.'  # change if files are elsewhere
TRAIN_PATH = os.path.join(DATA_DIR, 'Train.csv')
TEST_PATH = os.path.join(DATA_DIR, 'Test.csv')
OUT_DIR = os.path.join(DATA_DIR, 'artifacts')
os.makedirs(OUT_DIR, exist_ok=True)

# Options
USE_SMOTE = True        # try to use SMOTE; will fall back if imblearn not installed
RANDOM_STATE = 42
TOP_K = 15

# %% [markdown]
# ### Step 1 — Load data and quick inspection

# %%
# Load
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
print('Train shape:', train.shape)
print('Test shape :', test.shape)

# Quick head
print(train.head())


Train shape: (40000, 41)
Test shape : (10000, 41)
         V1        V2        V3        V4        V5        V6        V7  \
0 -4.464606 -4.679129  3.101546  0.506130 -0.221083 -2.032511 -2.910870   
1 -2.909996 -2.568662  4.109032  1.316672 -1.620594 -3.827212 -1.616970   
2  4.283674  5.105381  6.092238  2.639922 -1.041357  1.308419 -1.876140   
3  3.365912  3.653381  0.909671 -1.367528  0.332016  2.358938  0.732600   
4 -3.831843 -5.824444  0.634031 -2.418815 -1.773827  1.016824 -2.098941   

         V8        V9       V10  ...       V32       V33       V34       V35  \
0  0.050714 -1.522351  3.761892  ...  3.059700 -1.690440  2.846296  2.235198   
1  0.669006  0.387045  0.853814  ... -3.782686 -6.823172  4.908562  0.481554   
2 -9.582412  3.469504  0.763395  ... -3.097934  2.690334 -1.643048  7.566482   
3 -4.332135  0.565695 -0.101080  ... -1.795474  3.032780 -2.467514  1.894599   
4 -3.173204 -2.081860  5.392621  ... -0.257101  0.803550  4.086219  2.292138   

        V36       

In [2]:
# %% [markdown]
# ### Step 2 — Binarize target and examine class imbalance

# %%
if 'Target' not in train.columns:
    raise ValueError("Train.csv must contain a 'Target' column.")

# Binarize: non-zero -> 1
train['Target_bin'] = (train['Target'] != 0).astype(int)

print('Original Target unique values:', sorted(train['Target'].unique()))
print('Original binary counts:', train['Target_bin'].value_counts().to_dict())


Original Target unique values: [np.int64(0), np.int64(1)]
Original binary counts: {0: 37813, 1: 2187}



## Step 3 — Handle missing values (median imputation)


In [4]:
feature_cols = [c for c in train.columns if c.startswith('V')]
if len(feature_cols) == 0:
    raise ValueError('No feature columns found starting with "V"')

# Fill NaNs with median (train medians)
medians = train[feature_cols].median()
train[feature_cols] = train[feature_cols].fillna(medians)
test[feature_cols] = test[feature_cols].fillna(medians)



## Step 4 — Train/validation stratified split


In [5]:

# %%
X = train[feature_cols].copy()
y = train['Target_bin'].copy()

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

print('Train distribution before resample:', Counter(y_train))
print('Val distribution:', Counter(y_val))


Train distribution before resample: Counter({0: 30250, 1: 1750})
Val distribution: Counter({0: 7563, 1: 437})


## Step 5 — Balance training set (SMOTE preferred, fallback oversampling)


In [11]:
from imblearn.over_sampling import SMOTE

In [12]:

# %%
X_train_res = X_train.copy()
y_train_res = y_train.copy()

if USE_SMOTE:
    try:
        from imblearn.over_sampling import SMOTE
        sm = SMOTE(random_state=RANDOM_STATE)
        X_train_res, y_train_res = sm.fit_resample(X_train_res, y_train_res)
        print('SMOTE applied. New train distribution:', Counter(y_train_res))
    except Exception as e:
        print('SMOTE not available or failed:', e)
        # fallback to simple upsampling

# simple random upsampling fallback if SMOTE not applied or disabled
if Counter(y_train_res)[0] != Counter(y_train_res)[1]:
    cnt = Counter(y_train_res)
    maj = 0 if cnt[0] > cnt[1] else 1
    minc = 1 - maj
    idx_maj = y_train_res[y_train_res==maj].index
    idx_min = y_train_res[y_train_res==minc].index
    rng = np.random.RandomState(RANDOM_STATE)
    resample_idx = rng.choice(idx_min, size=(len(idx_maj)-len(idx_min)), replace=True)
    X_train_res = pd.concat([X_train_res.loc[idx_maj], X_train_res.loc[idx_min], X_train_res.loc[resample_idx]])
    y_train_res = pd.concat([y_train_res.loc[idx_maj], y_train_res.loc[idx_min], y_train_res.loc[resample_idx]])
    print('After simple oversampling train counts:', Counter(y_train_res))

# shuffle
X_train_res = X_train_res.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)
y_train_res = y_train_res.loc[X_train_res.index].reset_index(drop=True)


SMOTE applied. New train distribution: Counter({0: 30250, 1: 30250})


In [13]:


# %% [markdown]
# ### Step 6 — Feature selection (Mutual Information + XGBoost if available)

# %%
# Mutual information (nonparametric, works for continuous features)
mi = mutual_info_classif(X_train_res, y_train_res, random_state=RANDOM_STATE)
mi_series = pd.Series(mi, index=feature_cols).sort_values(ascending=False)
print('Top features by mutual info:\n', mi_series.head(10))

# Try XGBoost feature importances
xgb_available = True
try:
    import xgboost as xgb
    xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=RANDOM_STATE, verbosity=0)
    xgb_clf.fit(X_train_res, y_train_res)
    xgb_imp = pd.Series(xgb_clf.feature_importances_, index=feature_cols).sort_values(ascending=False)
    print('Top features by XGBoost importance:\n', xgb_imp.head(10))
except Exception as e:
    xgb_available = False
    xgb_imp = pd.Series(0, index=feature_cols)
    print('XGBoost unavailable or failed to run:', e)

# Combine ranks (average of MI rank and XGB rank) to get robust ranking
mi_rank = mi_series.rank(ascending=False)
xgb_rank = xgb_imp.rank(ascending=False)
combined_rank = ((mi_rank + xgb_rank) / 2.0).sort_values()

top_features = combined_rank.index[:TOP_K].tolist()
print(f'Top {TOP_K} features selected:', top_features)


Top features by mutual info:
 V28    0.004239
V38    0.004055
V36    0.003871
V37    0.003826
V8     0.002498
V5     0.002213
V39    0.001988
V3     0.001720
V34    0.001261
V21    0.001183
dtype: float64
Top features by XGBoost importance:
 V40    0.028686
V8     0.027415
V21    0.027272
V24    0.027190
V27    0.026182
V18    0.026170
V26    0.026134
V35    0.026121
V22    0.026043
V32    0.026020
dtype: float32
Top 15 features selected: ['V8', 'V21', 'V36', 'V37', 'V27', 'V38', 'V35', 'V18', 'V22', 'V30', 'V5', 'V40', 'V34', 'V16', 'V24']


In [14]:

# %% [markdown]
# ### Step 7 — Train model (RandomForest) on top features

# %%
# Train RandomForest as a robust baseline
rf = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=RANDOM_STATE)
rf.fit(X_train_res[top_features], y_train_res)

# Validation
y_val_pred = rf.predict(X_val[top_features])
if hasattr(rf, 'predict_proba'):
    y_val_prob = rf.predict_proba(X_val[top_features])[:,1]
else:
    y_val_prob = None

print('Validation classification report:')
print(classification_report(y_val, y_val_pred))
cm = confusion_matrix(y_val, y_val_pred)
print('Confusion matrix:\n', cm)

# PR-AUC
if y_val_prob is not None:
    precision, recall, _ = precision_recall_curve(y_val, y_val_prob)
    pr_auc = auc(recall, precision)
    print('PR-AUC:', pr_auc)

Validation classification report:
              precision    recall  f1-score   support

           0       0.94      0.52      0.67      7563
           1       0.05      0.45      0.09       437

    accuracy                           0.52      8000
   macro avg       0.50      0.48      0.38      8000
weighted avg       0.89      0.52      0.64      8000

Confusion matrix:
 [[3939 3624]
 [ 242  195]]
PR-AUC: 0.048632259642445094


In [15]:

# %% [markdown]
# ### Step 8 — Save model, scaler (optional) and test predictions

# %%
# Save model
# joblib.dump(rf, os.path.join(OUT_DIR, 'final_model.joblib'))

# Predict on test set (if test has no Target)
if set(feature_cols).issubset(set(test.columns)):
    test_preds = rf.predict(test[top_features])
    test_probs = rf.predict_proba(test[top_features])[:,1] if hasattr(rf, 'predict_proba') else None
    test_out = test.copy()
    test_out['pred'] = test_preds
    if test_probs is not None:
        test_out['pred_prob'] = test_probs
    test_out.to_csv(os.path.join(OUT_DIR, 'test_predictions.csv'), index=False)
    print('Saved test predictions to', os.path.join(OUT_DIR, 'test_predictions.csv'))
else:
    print('Test file does not contain expected feature columns; skipping test predictions')


Saved test predictions to .\artifacts\test_predictions.csv


In [ ]:
# %% [markdown]
# ### Step 10 — (Optional) SHAP explanations per prediction (local attribution)
#
# SHAP is the recommended way to show, for each faulty prediction, which features
# pushed the model toward the fault prediction. This block runs only if `shap`
# is installed. You can skip if not available.

# %%
try:
    import shap
    # Use TreeExplainer for tree models
    explainer = shap.TreeExplainer(rf)
    # compute on validation set (top features)
    shap_values = explainer.shap_values(X_val[top_features])
    # Example: get top 3 positive contributors for the first faulty prediction
    idxs_faulty = np.where((rf.predict(X_val[top_features]) == 1) & (y_val == 1))[0]
    if len(idxs_faulty) > 0:
        i = idxs_faulty[0]
        sv = shap_values[1][i] if isinstance(shap_values, list) else shap_values[i]
        contribs = pd.Series(sv, index=top_features).sort_values(ascending=False)
        print('Top 3 contributing features for one faulty instance:')
        print(contribs.head(3))
        # Save a SHAP force plot HTML
        shap_html = os.path.join(OUT_DIR, 'shap_force_example.html')
        shap.force_plot(explainer.expected_value[1], sv, X_val[top_features].iloc[i], matplotlib=False, show=False, out_names=None)
        # Note: shap.force_plot HTML saving requires shap.plots._force.manual
    else:
        print('No faulty instances in validation predictions to demo SHAP')
except Exception as e:
    print('shap not available or failed:', e)

In [ ]:


# %% [markdown]
# ### Step 9 — Visualizations (save PNGs)

# %%
# Class distributions
plt.figure(figsize=(6,4))
orig_counts = train['Target_bin'].value_counts()
plt.bar(['0 (normal)', '1 (fault)'], [orig_counts.get(0,0), orig_counts.get(1,0)])
plt.title('Original class distribution')
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, 'class_dist_original.png'))
plt.close()

plt.figure(figsize=(6,4))
train_counts_after = Counter(y_train_res)
plt.bar(['0 (normal)', '1 (fault)'], [train_counts_after.get(0,0), train_counts_after.get(1,0)])
plt.title('Training distribution after balancing')
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, 'class_dist_after_balancing.png'))
plt.close()

# Top features bar chart (combined rank values)
plt.figure(figsize=(8,6))
scores = combined_rank.loc[top_features].sort_values(ascending=True)
plt.barh(scores.index, scores.values)
plt.title('Top features (combined rank)')
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, 'top_features_combined_rank.png'))
plt.close()

# PCA on top features
pca = PCA(n_components=2, random_state=RANDOM_STATE)
X_pca = pca.fit_transform(pd.concat([X_train_res[top_features], X_val[top_features]]))
y_pca = pd.concat([y_train_res, y_val])
plt.figure(figsize=(7,6))
for cls in sorted(y_pca.unique()):
    mask = (y_pca == cls).values
    plt.scatter(X_pca[mask,0], X_pca[mask,1], label=str(cls), s=10, alpha=0.6)
plt.legend(title='Class')
plt.title('PCA (2D) of Top Features')
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, 'pca_top_features.png'))
plt.close()

# Confusion matrix heatmap
plt.figure(figsize=(5,4))
plt.imshow(cm, interpolation='nearest', cmap='viridis')
plt.title('Confusion Matrix (Validation)')
plt.colorbar()
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.xticks([0,1])
plt.yticks([0,1])
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, 'confusion_matrix.png'))
plt.close()

print('Saved visualizations to', OUT_DIR)



# %% [markdown]
# ### End — summary of saved artifacts

# %%
print('\nArtifacts saved in', OUT_DIR)
for fn in os.listdir(OUT_DIR):
    print('-', fn)

print('\nTop features list saved in variable top_features')
print('To run this notebook interactively: open this file in Jupyter/VSCode or convert to .ipynb with jupytext.')
